In [1]:
import PyPDF2
import csv
import pandas as pd
import openpyxl

##### PDF Crawling :)

In [2]:
text_pdf = []
# Open the PDF file in binary mode (Main documentation)
with open('../Stuff/VG_Variable.pdf', 'rb') as f:
    # Create a PdfFileReader object to read the PDF
    pdf = PyPDF2.PdfReader(f)
    # Identify the page numbers that contain the variables
    variable_pages = []
    for page in range(len(pdf.pages)):
        # Extract text from the page
        text = pdf.pages[page].extract_text()
        text_pdf.append(text)

In [3]:
all_text = ""
for page in text_pdf:
    all_text = all_text + "\n" + page
# print(all_text)

In [4]:
var_dict = {}
for section in all_text.split("____________________________________________________________________________________"):
    # print(section)
    before_var = None
    after_category = False
    cat_ans = False
    cats=["N", "Min", "Max", "Mean", "Std Dev"]
    for line in section.split("\n"):
        # print(line)
        if line.strip().__len__() == 0:
            continue
        if after_category:
            category = line
            after_category = False
            continue
        # print(line, "Release" in line)
        if "Variable Name:" in line:
            var_name = line.split("Variable Name:")[-1].strip()
            title = before_var
        elif "Label:" in line:
            label = line.split("Label:")[-1].strip()
        elif "SAS Dataset:" in line:
            ds = line.split("SAS Dataset:")[-1].strip()
        elif "Category" in line:
            after_category = True
        elif "NMin Max Mean Std Dev" == line.strip():
            cat_ans=True
        elif cat_ans:
            cat_ans = False
            ans = line.strip().split(" ")
            answer_of_categoris = {c: a for c, a in zip(cats, ans)}
            # print(answer_of_categoris)
        before_var = line
    var_dict[var_name] = {"Follow up": title, "Label": label, "Dataset_name":ds, "Category, Subcategory": category}

    var_dict[var_name].update(answer_of_categoris)
    

##### Dataset you need that:

In [5]:
df = pd.read_csv('..')
# remove ID and VERSION
allClinical_target_columns = df.columns.values[2:]

In [10]:
all_clinical = {}
for c_name in allClinical_target_columns:
    if c_name in var_dict:
        all_clinical[c_name] = var_dict[c_name]

In [11]:
print("len all_clinical target: ", len(allClinical_target_columns))
print("len all_clinical:", len(all_clinical))

len all_clinical target:  59
len all_clinical: 58


In [12]:
# finding difference between allClinical_target_columns and all_clinical
# which variables are not in main documentation
x = set(all_clinical.keys()).symmetric_difference(set(allClinical_target_columns))

In [14]:
x

{'VERSION'}

##### excel:

In [15]:
def create_excel_file(data):
    workbook = openpyxl.Workbook()
    sheet = workbook.active

    # Write headers
    headers = list(data.values())[0].keys()
    sheet.append(['Variable'] + list(headers))

    # Write data
    for key, values in data.items():
        row = [key] + list(values.values())
        sheet.append(row)

    # Save the workbook
    workbook.save('../documentation.xlsx')

# Example dictionary
data = all_clinical

# Create the Excel file
create_excel_file(data)